In [ ]:
pip install keras_applications

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 KB 5.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Concatenate, GlobalAveragePooling2D, Dense, Input, Lambda
import numpy as np
import pandas as pd
from keras_applications.resnet import ResNet50, preprocess_input

# from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import seaborn as sns
import cv2
import random
import os
import imageio
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
# from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, RepeatedStratifiedKFold
from imblearn.over_sampling import SMOTE

import tensorflow as tf
#import tensorflow_addons as tfa
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.applications import resnet
#from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
from keras.applications.resnet import ResNet50
# from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Input, Lambda, Dense, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


In [ ]:
from tensorflow.keras.applications import MobileNetV2


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
def load_images_labels(apple_dir, banana_dir, img_size):
    images = []
    labels = []
    
    for img_name in os.listdir(apple_dir):
        img_path = os.path.join(apple_dir, img_name)
        img = load_img(img_path, target_size=(img_size, img_size))
        img_array = img_to_array(img)
        images.append(preprocess_input(img_array))
        labels.append(0)  # Apple

    for img_name in os.listdir(banana_dir):
        img_path = os.path.join(banana_dir, img_name)
        img = load_img(img_path, target_size=(img_size, img_size))
        img_array = img_to_array(img)
        images.append(preprocess_input(img_array))
        labels.append(1)  # Banana

    return np.array(images), np.array(labels)

apple_dir = "/content/drive/MyDrive/mobilenet/apple vs banana/original_data_set/freshapples"
banana_dir = "/content/drive/MyDrive/mobilenet/apple vs banana/original_data_set/freshbanana"
img_size = 224

X, y = load_images_labels(apple_dir, banana_dir, img_size)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Define the input tensor
input_tensor = tf.keras.Input(shape=(224, 224, 3))

# Convert RGB to HSV
hsv_tensor = Lambda(lambda x: tf.image.rgb_to_hsv(x))(input_tensor)

# Convert RGB to YCbCr
ycbcr_tensor = Lambda(lambda x: tf.image.rgb_to_yuv(x))(input_tensor)

# Pass RGB, HSV, and YCbCr images through the MobileNet model
rgb_output = model_mobilenet(input_tensor)
hsv_output = model_mobilenet(hsv_tensor)
ycbcr_output = model_mobilenet(ycbcr_tensor)

# Concatenate the outputs of the MobileNet model
concatenated_output = Concatenate()([rgb_output, hsv_output, ycbcr_output])

# Add a global average pooling layer
global_average_pooling = GlobalAveragePooling2D()(concatenated_output)

dense = tf.keras.layers.Dense(128, activation='relu')(global_average_pooling)

# Add a binary classification layer
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

# Define the final model
model = tf.keras.models.Model(inputs=input_tensor, outputs=output_layer)

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 lambda (Lambda)                (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 lambda_1 (Lambda)              (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 mobilenetv2_1.00_224 (Function  (None, 7, 7, 1280)  2257984     ['input_2[0][0]',            

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train, epochs=200, validation_data=[X_test,y_test],batch_size=16)

Epoch 1/200
23/23 [==============================] - 6s 219ms/step - loss: 0.1511 - accuracy: 0.9722 - val_loss: 9.2776 - val_accuracy: 0.5667
Epoch 2/200
23/23 [==============================] - 4s 191ms/step - loss: 0.0619 - accuracy: 0.9833 - val_loss: 14.9510 - val_accuracy: 0.5111
Epoch 3/200
23/23 [==============================] - 5s 211ms/step - loss: 0.0459 - accuracy: 0.9889 - val_loss: 9.5380 - val_accuracy: 0.7000
Epoch 4/200
23/23 [==============================] - 4s 192ms/step - loss: 0.1381 - accuracy: 0.9611 - val_loss: 2.5988 - val_accuracy: 0.8333
Epoch 5/200
23/23 [==============================] - 5s 207ms/step - loss: 0.0366 - accuracy: 0.9861 - val_loss: 1.1721 - val_accuracy: 0.8778
Epoch 6/200
23/23 [==============================] - 5s 211ms/step - loss: 0.0113 - accuracy: 0.9972 - val_loss: 5.2760 - val_accuracy: 0.7556
Epoch 7/200
23/23 [==============================] - 4s 194ms/step - loss: 0.2700 - accuracy: 0.9722 - val_loss: 0.9216 - val_accuracy: 0.788

In [ ]:
pred=model.predict(X_test)

3/3 [==============================] - 0s 127ms/step


In [ ]:
pred=np.round(pred)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score


In [ ]:
conf_matrix = confusion_matrix(y_test,pred)
print(conf_matrix)

[[49  0]
 [ 0 41]]


In [ ]:
accuracy_score(y_test,pred)

1.0